## Exercise results on i7-10875H

In [269]:
%%file tmp.precision.cpp

#include <iostream>
#include <cassert> // for assert
#include <cstdlib> // for rand
#include <valarray>
#include <stdfloat>
#include <complex>
#include <cmath>

// SoA of complex numbers
template< typename R >
class Complexes {
  public :  
  
    Complexes( std::size_t size ) : m_rs(size), m_is(size) {}
    std::size_t size() const { return m_rs.size() ; }
    
    std::complex<R> operator[]( std::size_t index ) const
     { return { m_rs[index], m_is[index] } ; }
    void real( std::size_t index, R value ) { m_rs[index] = value ; }
    void imag( std::size_t index, R value ) { m_is[index] = value ; }
    
    friend Complexes<R> operator*( Complexes<R> const & lhs, Complexes<R> const & rhs ) {
      Complexes<R> res {lhs.size()} ;
      res.m_rs = lhs.m_rs*rhs.m_rs - lhs.m_is*rhs.m_is ;
      res.m_is = rhs.m_rs*lhs.m_is + lhs.m_rs*rhs.m_is ;
      return res ;
    }
    
  private :
    std::valarray<R> m_rs, m_is ;
 } ;

template< typename R >
Complexes<R> random( std::size_t size )
 {
  srand(1) ;
  Complexes<R> cplxs {size} ;
  for ( std::size_t i = 0 ; i < cplxs.size() ; ++i )
   {
    long double e = 2*M_PI*(static_cast<long double>(std::rand())/RAND_MAX) ;
    cplxs.real(i,static_cast<R>(std::cos(e))) ;
    cplxs.imag(i,static_cast<R>(std::sin(e))) ;
   }
  return cplxs ;
 }

template< typename R>
Complexes<R> pow( Complexes<R> const & cplxs, long long degree )
 {
  Complexes<R> res {cplxs} ;
  for ( long long d = 1 ; d < degree ; ++d ) {
    res = res*cplxs ;
  }
  return res ;
 }

template< typename R>
std::complex<R> reduce( Complexes<R> const & cplxs )
 {
  std::complex<R> acc { static_cast<R>(1.), static_cast<R>(0.) } ;
  for ( std::size_t i = 0 ; i < cplxs.size() ; ++i )
   { acc *= cplxs[i] ; }
  return acc ;
 }

template< typename R>
void main_impl( std::size_t size, long long degree )
 {
  std::complex<R> res = reduce(pow(random<R>(size),degree)) ;
  R re = res.real(), im = res.imag() ;
  auto r = std::sqrt(static_cast<long double>(re*re+im*im)) ;
  auto n = std::atan(static_cast<long double>(im/re)) ;
  std::cout<<r<<" "<<n<<std::endl ;
 }

int main( int argc, char * argv[] )
 {
  assert(argc==4) ;
  std::string precision(argv[1]) ;
  std::size_t size = atoi(argv[2]) ;
  long long degree = atoll(argv[3]) ;
  std::cout.precision(18) ;

  if (precision=="half") main_impl<std::float16_t>(size,degree) ;
  else if (precision=="float") main_impl<float>(size,degree) ;
  else if (precision=="double") main_impl<double>(size,degree) ;
  else if (precision=="long") main_impl<long double>(size,degree) ;
  else if (precision=="quad") main_impl<std::float128_t>(size,degree) ;
  else throw "unknown precision" ;
 }

Overwriting tmp.precision.cpp


In [281]:
%%file tmp.precision.sh
echo

opt=${1}
shift

rm -f tmp.precision.exe
g++ -O${opt} -Wconversion -march=native -std=c++23 tmp.precision.cpp -o tmp.precision.exe

./tmp.precision.exe $*

rm -f tmp.precision.py
echo "s = 0 ; m = 0" >> tmp.precision.py
for i in 0 1 2 3 4 5 6 7 8 9 ; do
    \time -f "s += %U ; m += %M" -a -o ./tmp.precision.py ./tmp.precision.exe $*  >> /dev/null
done
echo "print(\"{:.4} s. {} kB.\".format(s/10.,m/10.))" >> tmp.precision.py
python3 tmp.precision.py

echo

Overwriting tmp.precision.sh


In [282]:
! bash -l tmp.precision.sh 0 half 1024 10000


0 -nan
0.556 s. 3942.8 kB.



In [272]:
! bash -l tmp.precision.sh 0 float 1024 10000


1.00019518616282033 -0.119005833216005102
0.481 s. 3918.0 kB.



In [273]:
! bash -l tmp.precision.sh 0 double 1024 10000


1.00000000001092559 -0.120400202952363668
0.528 s. 3946.8 kB.



In [274]:
! bash -l tmp.precision.sh 0 long 1024 10000


1.00000000000000007 -0.120400202947234851
0.511 s. 4021.6 kB.



In [275]:
! bash -l tmp.precision.sh 0 quad 1024 10000


1.00000000000000009 -0.120400202947234855
1.897 s. 4057.6 kB.



### With `g++ -O0 ...`

| Type   | time (s) | significant digits |
| :------| -------: | -----------------: |
| half   |    0.545 |                  0 |
| float  |    0.481 |                  4 |
| double |    0.483 |                 11 |
| long   |    0.509 |                 17 |
| quad   |    1.819 |                 17 |


In [276]:
! bash -l tmp.precision.sh 1 half 1024 10000


0 -nan
0.099 s. 3901.2 kB.



In [277]:
! bash -l tmp.precision.sh 1 float 1024 10000


1.00019518616282033 -0.119005833216005102
0.018 s. 3929.2 kB.



In [278]:
! bash -l tmp.precision.sh 1 double 1024 10000


1.00000000001092559 -0.120400202952363668
0.021 s. 3933.6 kB.



In [279]:
! bash -l tmp.precision.sh 1 long 1024 10000


1.00000000000000007 -0.120400202947234851
0.056 s. 4000.4 kB.



In [280]:
! bash -l tmp.precision.sh 1 quad 1024 10000


1.00000000000000009 -0.120400202947234855
1.351 s. 4006.0 kB.



#### With `g++ -O1 ...`

| Type   | time (s) | significant digits |
| :------| -------: | -----------------: |
| half   |    0.094 |                  0 |
| float  |    0.018 |                  4 |
| double |    0.020 |                 11 |
| long   |    0.066 |                 18 |
| quad   |    1.224 |                 18 |

In [239]:
! bash -l tmp.precision.sh 2 half 1024 10000


0
0.094 s. 3948.8 kB.



In [247]:
! bash -l tmp.precision.sh 2 float 1024 10000


0.999907936388290119
0.01 s. 3956.4 kB.



In [241]:
! bash -l tmp.precision.sh 2 double 1024 10000


1.00000000001088041
0.019 s. 3919.2 kB.



In [242]:
! bash -l tmp.precision.sh 2 long 1024 10000


1
0.061 s. 4012.8 kB.



In [243]:
! bash -l tmp.precision.sh 2 quad 1024 10000


1
1.273 s. 4050.0 kB.



#### With `g++ -O2 ...`

| Type   | time (s) | significant digits |
| :------| -------: | -----------------: |
| half   |    0.094 |                  0 |
| float  |    0.010 |                  4 |
| double |    0.019 |                 11 |
| long   |    0.061 |                 18 |
| quad   |    1.273 |                 18 |

In [252]:
! bash -l tmp.precision.sh 3 half 1024 100000


-nan
0.977 s. 3902.4 kB.



In [253]:
! bash -l tmp.precision.sh 3 float 1024 100000


0.99935391535653062
0.053 s. 3882.8 kB.



In [254]:
! bash -l tmp.precision.sh 3 double 1024 100000


1.00000000010806178
0.141 s. 3970.8 kB.



In [255]:
! bash -l tmp.precision.sh 3 long 1024 100000


1.00000000000000089
0.622 s. 3996.4 kB.



In [256]:
! bash -l tmp.precision.sh 3 quad 1024 100000


1.00000000000000089
12.73 s. 3976.8 kB.



#### With `g++ -O3 ...`

| Type   | time (s) | significant digits |
| :------| -------: | -----------------: |
| half   |    0.959 |                nan |
| float  |    0.053 |                  3 |
| double |    0.141 |                 10 |
| long   |    0.622 |                 16 |
| quad   |   12.73  |                 16 |


In [257]:
! bash -l tmp.precision.sh fast half 1024 100000


-nan
0.95 s. 3620.0 kB.



In [258]:
! bash -l tmp.precision.sh fast float 1024 100000


0.99935391535653062
0.054 s. 3684.0 kB.



In [259]:
! bash -l tmp.precision.sh fast double 1024 100000


1.00000000010791479
0.122 s. 3713.6 kB.



In [260]:
! bash -l tmp.precision.sh fast long 1024 100000


1.00000000000003797
0.668 s. 3788.4 kB.



In [262]:
! bash -l tmp.precision.sh fast quad 1024 100000


1.00000000000003819
12.17 s. 3807.2 kB.



#### With `g++ -Ofast ...`

| Type   | time (s) | significant digits |
| :------| -------: | -----------------: |
| half   |    0.95  |                nan |
| float  |    0.054 |                  3 |
| double |    0.122 |                 10 |
| long   |    0.668 |                 14 |
| quad   |   12.17  |                 14 |


© *CNRS 2024*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*